<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#parameters" data-toc-modified-id="parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>parameters</a></span><ul class="toc-item"><li><span><a href="#encoder-size" data-toc-modified-id="encoder-size-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>encoder size</a></span></li><li><span><a href="#encoders-needed:-5?" data-toc-modified-id="encoders-needed:-5?-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>encoders needed: 5?</a></span><ul class="toc-item"><li><span><a href="#optional-strat:-drop-non-normalized-SMART-stats-and-try-again-(if-runtime-too-long),-since-we're-sort-of...-double-encoding,-almost" data-toc-modified-id="optional-strat:-drop-non-normalized-SMART-stats-and-try-again-(if-runtime-too-long),-since-we're-sort-of...-double-encoding,-almost-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>optional strat: drop non-normalized SMART stats and try again (if runtime too long), since we're sort of... double encoding, almost</a></span></li></ul></li></ul></li><li><span><a href="#class-imbalance-easing" data-toc-modified-id="class-imbalance-easing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>class imbalance easing</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#quicker-pre-indexed-method" data-toc-modified-id="quicker-pre-indexed-method-2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>quicker pre-indexed method</a></span></li></ul></li></ul></li><li><span><a href="#failure-horizons-complete" data-toc-modified-id="failure-horizons-complete-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>failure horizons complete</a></span></li></ul></div>

In [70]:
import csv
import datetime
import os
import numpy as np
import random
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm, tqdm_notebook

from htm.bindings.sdr import SDR, Metrics
from htm.encoders.rdse import RDSE, RDSE_Parameters
from htm.encoders.date import DateEncoder
from htm.encoders import scalar_encoder
from htm.encoders.scalar_encoder import ScalarEncoder, ScalarEncoderParameters
from htm.bindings.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood #FIXME use TM.anomaly instead, but it gives worse results than the py.AnomalyLikelihood now
from htm.bindings.algorithms import Predictor

In [97]:
df = pd.read_csv('hard_disks.csv', 
                 index_col=0
                )
# df = df.drop(columns=['Unnamed: 0'])
print('reorganizing for "lifetime" view...')
df = df.sort_values(by=['date','serial_number']) # should it be serial_num then date? i wonder
print('cleaning indices...')
df = df.reset_index()
df.drop(columns=['index'], inplace=True)

len(df.columns)

reorganizing for "lifetime" view...
cleaning indices...


61

# parameters
## encoder size
encoder 'size' can be small since we're running 62 columns

In [ ]:
parameters = {
  # won't be using these exact params for each encoder, however.
 'enc': {
      "value" :
         {'resolution': 0.88, 'size': 800, 'sparsity': 0.02},
      "time": 
         {'timeOfDay': (30, 1), 'weekend': 21}
 },
 'predictor': {'sdrc_alpha': 0.1},
 'sp': {'boostStrength': 3.0,
        'columnCount': 3200, # tweak this a little
        'localAreaDensity': 0.04395604395604396,
        'potentialPct': 0.85,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'synPermInactiveDec': 0.006},
 'tm': {'activationThreshold': 17,
        'cellsPerColumn': 20,
        'initialPerm': 0.21,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'minThreshold': 10,
        'newSynapseCount': 32,
        'permanenceDec': 0.1,
        'permanenceInc': 0.1},
 'anomaly': {
   'likelihood': 
       {#'learningPeriod': int(math.floor(self.probationaryPeriod / 2.0)),
        #'probationaryPeriod': self.probationaryPeriod-default_parameters["anomaly"]["likelihood"]["learningPeriod"],
        'probationaryPct': 0.1,
        'reestimationPeriod': 100} #These settings are copied from NAB
 }
}

## encoders needed: 5?
- dateTime encoder to understand temporal sequence element
- category encoder...? for the model
    - could just convert each model to a number
    - maybe RDSE does this for us, we'll see
- scalar for capacity_bytes
- one-bit encoder for 'failure' tacked on at the end, maybe.
- SMART stat encoder

maybe 4 encoders, we can use SMART_enc for capacity_bytes 

### optional strat: drop non-normalized SMART stats and try again (if runtime too long), since we're sort of... double encoding, almost

In [38]:
smarts = [col for col in list(df.columns) if 'smart' in col]

In [64]:
date_encoder = DateEncoder(timeOfDay=(30,1), weekend=21) # less work is done on weekends
# predict 'weekday vs weekend' HD consumption use to predict impending failure...?
# maybe unnecessary, we'll see

# SMART encoder
stat_params = RDSE_Parameters()
stat_params.size = 30 # this number is the one i'll be changing around the most
stat_params.sparsity = 0.02 # the magic number
stat_params.resolution = 0.88 
stat_encoder = RDSE(stat_params) # create encoder
# we could also use this for byte_capacity. model

# failure 1-bit encoder to minimize SDR size. we only need one bit to encode 1 or 0, after all
fail_params = ScalarEncoderParameters() 
fail_params.minimum = 0
fail_params.maximum = 1
fail_params.size = 30 # if i go under 30, it crashes: CHECK FAILED: "args_.activeBits > 0u" 
    # this is a pretty awful problem, considering i'm wasting 29 bits of calculation
    # i'm probably missing some configuration in a setting somewhere
fail_params.sparsity = 0.02
# fail_params.resolution = 0.88
fail_encoder = ScalarEncoder(fail_params) # it probably could just be a ScalarEncoder, not RDSE ...

# # 5-bit scalar encoder for categorical "model" string, identifies unique hard disk.
model_params = RDSE_Parameters() # in retrospect, naming one of the encoders 'model' isn't great nomenclature
model_params.size = 30 # 5! will take care of 45 unique models
model_params.sparsity = 0.02
model_params.resolution = 0.88
model_encoder = RDSE(model_params)

# serial num encoder. 145,000 uniques
serial_params = RDSE_Parameters() # in retrospect, naming one of the encoders 'model' isn't great nomenclature
serial_params.size = 30 # 5! will take care of 45 unique models
serial_params.sparsity = 0.02
serial_params.resolution = 0.88
serial_encoder = RDSE(serial_params)

# calculate total bits in encoder
# smart encoder is used 56 times (for SMART stats) and once for byte capacity
encoder_width = date_encoder.size + stat_encoder.size*(len(smarts)+1) + fail_encoder.size \
+ model_encoder.size + serial_encoder.size
encoder_info = Metrics([encoder_width], 999999999) # more 9s, the better



# i was making a separate encoder for capacity_bytes, but it's 14 zeros and 30 bits from
# STAT_encoder is already 10^32
# mem_params = RDSE_Parameters
# mem_params.size = 10 # max(df['capacity_bytes']) = 16000900661248. 14 zeros needs a deal of bits

### each disk has a serial_number serving as unique identifier
we've got 45 unique models and 144,876 unique hard_drives out of 12.5 million rows.

In [24]:
df.loc[df['failure']==1]

,date,serial_number,model,capacity_bytes,failure,smart_1_normalized,smart_1_raw,smart_2_normalized,smart_2_raw,smart_3_normalized,...,smart_199_normalized,smart_199_raw,smart_200_normalized,smart_200_raw,smart_240_normalized,smart_240_raw,smart_241_normalized,smart_241_raw,smart_242_normalized,smart_242_raw
39086,2020-04-01,S301NDVZ,ST4000DM000,4000787030016,1,117.0,142762888.0,NaN,NaN,91.0,...,200.0,0.0,NaN,NaN,100.0,38459.0,100.0,5.596416e+10,100.0,2.416296e+11
93536,2020-04-01,ZCH098ER,ST12000NM0007,12000138625024,1,75.0,34494144.0,NaN,NaN,90.0,...,200.0,0.0,100.0,0.0,100.0,19053.0,100.0,6.379076e+10,100.0,1.637831e+11
104410,2020-04-01,ZHZ3SS54,ST12000NM0008,12000138625024,1,69.0,93496976.0,NaN,NaN,97.0,...,200.0,0.0,100.0,0.0,100.0,3021.0,100.0,3.644411e+10,100.0,5.569063e+10
137352,2020-04-02,5641SFRFS,TOSHIBA MQ01ABF050,500107862016,1,100.0,0.0,100.0,0.0,100.0,...,200.0,0.0,NaN,NaN,100.0,0.0,NaN,NaN,NaN,NaN
194389,2020-04-02,ZA120V8D,ST8000NM0055,8001563222016,1,83.0,216744360.0,NaN,NaN,89.0,...,200.0,0.0,NaN,NaN,100.0,29234.0,100.0,7.230961e+10,100.0,1.917249e+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12077347,2020-06-28,57RFWNHJT,TOSHIBA MQ01ABF050,500107862016,1,100.0,0.0,100.0,0.0,100.0,...,200.0,0.0,NaN,NaN,100.0,0.0,NaN,NaN,NaN,NaN
12117280,2020-06-28,Z302DJW1,ST4000DM000,4000787030016,1,103.0,5679912.0,NaN,NaN,94.0,...,200.0,0.0,NaN,NaN,100.0,45380.0,100.0,6.669282e+10,100.0,2.049333e+11
12286683,2020-06-29,ZA14EBBG,ST8000NM0055,8001563222016,1,51.0,155947482.0,NaN,NaN,89.0,...,200.0,0.0,NaN,NaN,100.0,28651.0,100.0,6.762404e+10,100.0,2.168453e+11
12295966,2020-06-29,ZA1819EP,ST8000NM0055,8001563222016,1,55.0,32634152.0,NaN,NaN,91.0,...,200.0,0.0,NaN,NaN,100.0,25265.0,100.0,6.249917e+10,100.0,1.831933e+11


# class imbalance easing
- borrowing that guy's solution (in EDA notebook) of 'failure horizons'. if a failure occurs, append failure=1 to the past 7 days of that drive's rows. pre-sorting the drives by date & serial number helps here.
- iterating through the whole DF is computationally pricy
    - instead, collect failure indexes by subsetting where failure==1
        - append to list
        - loop through list, adding X-7, X-6... for every index X to create failure horizons
        - then loop through the list of indexes and use df.at[indx, colname] = value
            - this saves a lot of copy-warning headaches

### quicker pre-indexed method

In [102]:
fails = df[df['failure']==1]
inds = list(fails.index)
print('failure indexes collected')

horizons = [] 
for ind in inds: # indexes with current failures
    for b in range(-7, 0):
        horizons.append(b+ind) # append the last 7 days
    horizons.append(ind) # append the day
print('failure horizon indexes drawn')

failure horizon indexes drawn


In [107]:
(len(horizons)-314) / 7

314.0

In [111]:
for h in horizons:
    # df.at or df.set_value is even faster than df.ix
    df.at[h, 'failure'] = 1
print('final failure count: ' + str(len(df[df['failure']==1])))


final failure count: 2512


# failure horizons complete